In [2]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import os
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join('.', 'db.sqlite3'),
    }
}
import json


import chromadb

import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS
import os

config_list = [
    {"model": "gpt-3.5-turbo-0125", "api_key": os.getenv('OPENAI_API_KEYS'), "api_type": "openai"},
]

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-3.5-turbo-0125']


In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model=config_list[0]["model"], openai_api_key=os.getenv('OPENAI_API_KEYS'))

/workspaces/Contract-Advisor-RAG/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recur_spliter = RecursiveCharacterTextSplitter(separators=["\n", "\r", "\t"])

In [28]:
# 1. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="""
    You are an AI expert specialized in contract analysis and advisory services.
    Your goal is to assist users by providing precise and helpful answers based on a contract document.
    The document in question is an Advisory Services Agreement.
    Given the user query, generate a well-structured and informative response by extracting relevant information from the document.
    Ensure the response is concise, accurate, and directly addresses the query.
    """,
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
    },
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": [
            '../data/Robinson Advisory.docx.pdf'
        ],
        #"custom_text_types": ["non-existent-type"],
        "chunk_token_size": 1000,
        "model": config_list[0]["model"],
        #"client": chromadb.PersistentClient(path="/tmp/chromadb"),  # deprecated, use "vector_db" instead
        "vector_db": "chroma",  # to use the deprecated `client` parameter, set to None and uncomment the line above
        "overwrite": True,  # set to True if you want to overwrite an existing collection
        #"chunk_mode": "one_line",
        "must_break_at_empty_line": False,
        #"embedding_model": embed_model,
        "update_context": True,
        "custom_text_split_function": recur_spliter.split_text,
        #"custom_text_split_function": autogen.retrieve_utils.split_text_to_chunks(max_tokens = 1000, chunk_mode = "multi_lines", must_break_at_empty_line = True, overlap = 10)'
        
    },
    code_execution_config=False,  # set to False if you don't want to execute the code
)

In [21]:
assistant.reset()

ragproxyagent.retrieve_docs("Who are the parties to the Agreement and what are their defined names?", n_results=10)

Trying to create collection.


2024-07-06 07:07:36,737 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 6 chunks.
2024-07-06 07:07:36,740 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.
Number of requested results 10 is greater than number of elements in index 6, updating n_results = 6


VectorDB returns doc_ids:  [['cda0c86e', 'c287ed1b', 'f2b134ef', '3319ba92', 'd792e245', '1ea24cba']]


In [24]:
assistant.reset()

qa_problem = "What is the termination notice?"
#ragproxyagent.retrieve_docs(qa_problem, n_results=10)
chat_result = ragproxyagent.initiate_chat(assistant, 
                                          message=ragproxyagent.message_generator,
                                          problem=qa_problem,
                                          retrieval = ragproxyagent.retrieve_docs(qa_problem, n_results=10))

Trying to create collection.


2024-07-06 07:16:20,365 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 1 chunks.
2024-07-06 07:16:20,369 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 20 is greater than number of elements in index 1, updating n_results = 1


VectorDB returns doc_ids:  [['59ed6a14']]
VectorDB returns doc_ids:  [['59ed6a14']]
Skip doc_id 59ed6a14 as it is too long to fit in the context.
No more context, will terminate.
ragproxyagent (to assistant):

TERMINATE

--------------------------------------------------------------------------------


In [9]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "What is the termination notice?"
chat_result = ragproxyagent.initiate_chat(assistant, 
                                          message=ragproxyagent.message_generator,
                                          problem=qa_problem)
#                                          search_string="spark")

# assistant.reset()
# code_problem = "How can I use FLAML to perform a classification task and use spark to do parallel training. Train 50 seconds and force cancel jobs if time limit is reached."
# chat_result = ragproxyagent.initiate_chat(
#     assistant, message=ragproxyagent.message_generator, problem=code_problem
# ) 

Number of requested results 20 is greater than number of elements in index 4, updating n_results = 4


VectorDB returns doc_ids:  [['1a2229c0', 'ce401979', '051e575e', '4640dff1']]
Adding content of doc 1a2229c0 to context.
Adding content of doc ce401979 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What is the termination notice?

Context is: for
any
loss
or
expenses
incurred
by
the
Company
if
it
were
determined
that
an
alleged 
employer/employee
relationship
existed
between
the
Advisor
and
the
Company .
9.
Entir e
Agreement;
No
Waiver
or
Assignment
:
This
Agreement
together
with
the
Exhibits,
which 
are
attached
hereto
and
incorporated
herein,
set
forth
the
entire
Agreement
between
the
parties
and 
shall
supersede
all
previous
communications
and
agreements
between
the
parties,
eithe

In [75]:
assistant.reset()
userproxyagent = autogen.UserProxyAgent(name="userproxyagent")
userproxyagent.initiate_chat(assistant, message="What is the termination notice?")

userproxyagent (to assistant):

What is the termination notice?

--------------------------------------------------------------------------------


assistant (to userproxyagent):

The termination notice period specified in the Advisory Services Agreement is 30 days. This means either party can terminate the agreement by providing a written notice at least 30 days in advance.

--------------------------------------------------------------------------------
userproxyagent (to assistant):

 According to section 4:14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of this Agreement.  

--------------------------------------------------------------------------------
assistant (to userproxyagent):

In the Advisory Services Agreement, it is stated that both parties have the right to terminate the agreement for convenience by providing a 14-day notice. Additionally, the Company reserves the right to terminate the agreement without notice if the Advisor refuses or is unable to perform the services or breaches any provision 

KeyboardInterrupt: Interrupted by user